In [13]:
###CODE TO EXPLAINTHE SIMPLE IMPLEMENTATION 
import openai
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [14]:
# calling the api_key 
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")

else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'

openai = OpenAI()

API key looks good so far


Purpose: Some websites require proper HTTP headers to allow requests. The User-Agent header mimics a browser request, making the server think the request comes from a legitimate browser (like Chrome).

In [16]:
# Header configuration 
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


2. The Website Class
Purpose:
Represents a webpage and provides methods to extract key information, including:

Title of the webpage
Text content
Links present on the page
Initialization (__init__ Method)

In [20]:
class Website:
    """
    A utility class to represent a Website that we have scraped, now with links.
    This class fetches the content of a webpage, extracts its title, text content,
    and the hyperlinks present on the page.
    """

    def __init__(self, url):
        """
        Initializes the Website object by fetching and parsing the content of the given URL.
        
        Args:
            url (str): The URL of the webpage to scrape.
        """
        # Store the URL of the webpage
        self.url = url

        # Send an HTTP GET request to the URL with specified headers to mimic a real browser
        response = requests.get(url, headers=headers)
        
        # Store the raw HTML content of the webpage
        self.body = response.content

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(self.body, 'html.parser')

        # Extract the title of the webpage, or use a fallback if no title is found
        self.title = soup.title.string if soup.title else "No title found"

        # Extract the main text content of the webpage
        if soup.body:  # Ensure there is a <body> element in the HTML
            # Remove irrelevant elements like <script>, <style>, <img>, and <input> tags
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()  # Completely removes these tags from the parsed tree

            # Get the text content from the <body> tag, separating sections with newlines
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            # If no <body> tag is present, set text content to an empty string
            self.text = ""

        # Extract all hyperlinks (href attributes of <a> tags)
        links = [link.get('href') for link in soup.find_all('a')]

        # Filter out empty or invalid links
        self.links = [link for link in links if link]

    def get_contents(self):
        """
        Returns a formatted string containing the title and text content of the webpage.
        
        Returns:
            str: A string representation of the webpage's title and main content.
        """
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [21]:
ch = Website("https://chaayos.com/")

In [19]:
ch.links

['#MainContent',
 'https://www.instagram.com/chaayos/?hl=en',
 'https://www.facebook.com/Chaayos/',
 'https://twitter.com/chaayos?lang=en',
 '/',
 '/',
 '/collections',
 '/products/meri-wali-chai-patti',
 '/collections/instant-chai',
 '/collections/best-sellers',
 '/collections/new-arrivals',
 '/collections/desi-chai-patti',
 '/collections/chai-time-snacks',
 '/collections/gifting',
 '/collections/herbal-tea',
 '/collections/natural-chai-spices',
 '/collections/tea-bags',
 '/collections/green-tea',
 '/collections',
 '/pages/about-brand',
 '/pages/about-brand',
 '/pages/our-story',
 '/pages/safety',
 '/pages/cafe-locator',
 'https://docs.google.com/forms/d/11ofNaWkzm1OqUY812uGSnXSCMCSgfH-ouqmSFOIPDRA/viewform?edit_requested=true',
 '/search',
 '/account',
 '/cart',
 '/products/chaayos-instant-tea-masala-regular-sugar-36-sachets',
 '/products/meri-wali-chai-patti',
 '/products/masala-chai-patti-flavoured-black-tea',
 '/products/instant-tea-adrak-elaichi-low-sugar',
 '/account',
 '/produc

In [22]:
##Prompt for link Classification
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""


In [23]:
##User Specific prompt 
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt


In [24]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [25]:
print(get_links_user_prompt)

<function get_links_user_prompt at 0x0000028A4029C680>


In [26]:
print(get_links_user_prompt(ch))

Here is the list of links on the website of https://chaayos.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#MainContent
https://www.instagram.com/chaayos/?hl=en
https://www.facebook.com/Chaayos/
https://twitter.com/chaayos?lang=en
/
/
/collections
/products/meri-wali-chai-patti
/collections/instant-chai
/collections/best-sellers
/collections/new-arrivals
/collections/desi-chai-patti
/collections/chai-time-snacks
/collections/gifting
/collections/herbal-tea
/collections/natural-chai-spices
/collections/tea-bags
/collections/green-tea
/collections
/pages/about-brand
/pages/about-brand
/pages/our-story
/pages/safety
/pages/cafe-locator
https://docs.google.com/forms/d/11ofNaWkzm1OqUY812uGSnXSCMCSgfH-ouqmSFOIPDRA/viewform?edit_requested=true
/search
/account
/cart
/products/chaayos-instant-tea-masala-regula

In [28]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [29]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co") 
huggingface.links                                                                       

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/smolagents',
 '/microsoft/phi-4',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/DeepSeek-V3',
 '/black-forest-labs/FLUX.1-dev',
 '/NovaSky-AI/Sky-T1-32B-Preview',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/osanseviero/gemini-coder',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/reach-vb/2024-ai-timeline',
 '/spaces/fffiloni/LatentSync',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/DAMO-NLP-SG/multimodal_textbook',
 '/datasets/cfahlgren1/react-code-instructions',
 '/datasets/PowerInfer/QWQ-LONGCOT-500K',
 '/datasets/OpenLeecher/lmsys_chat_1m_clean',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '

In [30]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'company social', 'url': 'https://twitter.com/huggingface'},
  {'type': 'company social',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [31]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [32]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
Use Ollama with GGUF Models from the HF Hub
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
microsoft/phi-4
Updated
4 days ago
•
42.8k
•
1.02k
hexgrad/Kokoro-82M
Updated
6 days ago
•
9.72k
•
744
deepseek-ai/DeepSeek-V3
Updated
13 days ago
•
112k
•
1.69k
black-forest-labs/FLUX.1-dev
Updated
Aug 16, 2024
•
1.25M
•
7.93k
NovaSky-AI/Sky-T1-32B-Preview
Updated
1 day 

In [33]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [34]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [35]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nUse Ollama with GGUF Models from the HF Hub\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmicrosoft/phi-4\nUpdated\n4 days ago\n•\n42.8k\n•\n1.02k\nhexgrad/Kokoro-82M\nUpdated\n6 days ago\n•\n9.72k\n•\n744\ndeepseek-ai/DeepSeek-V3\nUpdated\n13 days ago\n•\n112k\n•\n1.69k\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16, 2024\n•\n1.25M\n•\n7.93k\nNovaSky-AI/Sky-T1-32B-Preview\nUpdated